In [87]:
import re
import tweepy
from textblob import TextBlob
import json
import codecs

In [3]:
BEARER_TOKEN = ""

In [111]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''

    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
#         # attempt authentication
#         try:
#             self.auth = tweepy.OAuth2BearerHandler(BEARER_TOKEN)

#             # create tweepy API object to fetch tweets
#             self.api = tweepy.API(self.auth)
#         except:
#             print("Error: Authentication Failed")
        self.client = tweepy.Client(bearer_token=BEARER_TOKEN)

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(
            re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count=50, lang="en"):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # call twitter api to fetch tweets
#             fetched_tweets = self.api.search_tweets(q=query, lang=lang, locale="usa", count=count, tweet_mode="extended")
            fetched_tweets = self.client.search_recent_tweets(query=query, 
                              tweet_fields=['public_metrics', 'context_annotations', 'entities', 'geo', 'lang', 'in_reply_to_user_id',
                                           'referenced_tweets', 'source', 'text', 'withheld', 'attachments'
                                           ],
                                expansions=['attachments.media_keys', 'referenced_tweets.id', 'author_id', 'geo.place_id'],
                                media_fields=['type', 'duration_ms', 'preview_image_url'],
                             )
            # parsing tweets one by one
            for tweet in fetched_tweets.data:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet
                parsed_tweet = tweet.data
#                 parsed_tweet['id'] = tweet.id
#                 parsed_tweet['text'] = tweet.text
#                 parsed_tweet['attachments'] = tweet.attachments
#                 parsed_tweet['public_metrics'] = tweet.public_metrics
# #                 parsed_tweet['referenced_tweets'] = tweet.referenced_tweets
#                 parsed_tweet['author_id'] = tweet.author_id
                
                tweets.append(parsed_tweet)

            # return parsed tweets
            return tweets

        except tweepy.TweepyException as e:
            # print error (if any)
            print("Error : " + str(e))

In [112]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query='(#pepsi OR #coke OR #cocacola) -is:retweet', count=100)
#     json.dumps(tweets, ensure_ascii=False).encode('utf8')
#     with codecs.open("test.json", 'w') as fp:
#         fp.write(json.dumps(tweets, ensure_ascii=False).encode('utf8'))
    with open('test.json', 'w', encoding='utf8') as json_file:
        json.dump(tweets, json_file, ensure_ascii=False, indent=2)

In [113]:
# calling main function
main()